<a href="https://colab.research.google.com/github/AlvinKimata/Deep-learning-in-Python-Datacamp-skill-track/blob/main/3_4_Training_convolutional_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Create the train and test loaders.

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, 
                                        download = True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 128, 
                                          shuffle = True, num_workers = 2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [3]:
testset = torchvision.datasets.CIFAR10(root = './data', train = False, 
                                      download = True, transform = transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = 128, 
                                         shuffle = False, num_workers = 2)

Files already downloaded and verified


## Building the CNN.

In [4]:
class Net(nn.Module):
    def __init__(self, num_classes = 10):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(128 * 4 * 4, num_classes) #(number of units from the last layer, 4 * 4 is height and width of image)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 128 * 4 * 4) #Squeeze all dimensions in one dimension (Flattening)
        
        return self.fc(x)
        

## Instantiate optimizer and loss function.

In [5]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 3e-4)

## Train the CNN.

In [6]:
#Train for 10 epochs.
for epoch in range(10):
    for i, data in enumerate(trainloader, 0):
        #Fetch the inputs.
        inputs, labels = data
        
        #Zero the parameter gradients.
        optimizer.zero_grad()
        
        #Forward + backward + optimize.
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        #Compute gradients.
        loss.backward()
        
        #Update weights.
        optimizer.step()
        
print('Finished training')

Finished training


## Evaluating results.

In [7]:
correct, total = 0, 0
predictions = []

#Put the net in evaluation mode.
net.eval()

for i, data in enumerate(testloader, 0):
  inputs, labels = data
  outputs = net(inputs)

  _, predicted = torch.max(outputs.data, 1)
  predictions.append(outputs)

  total += labels.size(0)
  correct += (predicted == labels).sum().item()


print('The test set accuracy of the network is: %d %%' %(100 * correct / total))

The test set accuracy of the network is: 70 %
